In [1]:
from numpy.core.multiarray import result_type
!wget -O "yolov3.weights" "https://github.com/patrick013/Object-Detection---Yolov3/blob/master/model/yolov3.weights?raw=true"
!wget -O "yolov3.cfg" "https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg?raw=true"
!wget -O "coco.names" "https://github.com/pjreddie/darknet/blob/master/data/coco.names?raw=true"

import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Status
Interaction=False

net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

# Load classes (coco.names contains the names of 80 classes)
with open("coco.names", "r") as f:
    classes = f.read().strip().split('\n')
# Reading Video
vidcap = cv2.VideoCapture('/video_dir')
success,image = vidcap.read()
count=0
model=keras.models.load_model('drive/MyDrive/HOI_model')
while success:
  clear_output()
  print(f"Interaction Detected at {round(count/30)} seconds") if Interaction else print(f"Interaction Not Detected at {round(count/30)} seconds")

  success,image = vidcap.read()
  count += 1
  if count%30==0:
     # Get image dimensions
    height, width = image.shape[:2]

    # Prepare the image for YOLO
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
    # Set input to the model
    net.setInput(blob)

    # Get output layer names
    output_layer_names = net.getUnconnectedOutLayersNames()

    # Run forward pass
    outs = net.forward(output_layer_names)

    # Initialize lists for detected objects' class IDs and confidence scores
    class_ids = []
    confidences = []
    # Post-processing: Get class IDs and confidence scores
    for out in outs:
      for detection in out:
          scores = detection[5:]
          class_id = np.argmax(scores)
          confidence = scores[class_id]

          if confidence > 0.7:  # Adjust this confidence threshold as needed
              if classes[class_id] == "person":
                  blob_reshaped=blob.reshape(3, 416, 416)
                  blob_transposed = np.transpose(blob_reshaped, (1, 2, 0))
                  final_image=blob_transposed.reshape(1, 416, 416, 3)
                  prob=model.predict(final_image).reshape(1)[0]
                  result = round(prob)
                  if result == 1:
                    Interaction=True
                    break
                  else :
                    Interaction=False
      if Interaction:
        break

--2023-12-14 09:50:34--  https://github.com/patrick013/Object-Detection---Yolov3/blob/master/model/yolov3.weights?raw=true
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/patrick013/Object-Detection---Yolov3/raw/master/model/yolov3.weights [following]
--2023-12-14 09:50:34--  https://github.com/patrick013/Object-Detection---Yolov3/raw/master/model/yolov3.weights
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/patrick013/Object-Detection---Yolov3/master/model/yolov3.weights [following]
--2023-12-14 09:50:34--  https://media.githubusercontent.com/media/patrick013/Object-Detection---Yolov3/master/model/yolov3.weights
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, 